Progress Report - Arima and Time Series
NYC Taxi Fare Prediction

In [1]:
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import itertools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
import warnings
warnings.filterwarnings("ignore")

In [2]:
nyc_data  = pd.read_csv(r"C:\Users\Hardik Patil\Downloads\new-york-city-taxi-fare-prediction\train.csv",nrows = 2000000)
nyc_data.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [3]:
coord = ['pickup_longitude','pickup_latitude', 
         'dropoff_longitude', 'dropoff_latitude']

for i in coord :
    nyc_data[i] = nyc_data[i].replace(0,np.nan)
    nyc_data    = nyc_data[nyc_data[i].notnull()]


nyc_data["pickup_datetime"] = nyc_data["pickup_datetime"].str.replace(" UTC","")
nyc_data["pickup_datetime"] = pd.to_datetime(nyc_data["pickup_datetime"],
                                             format="%Y-%m-%d %H:%M:%S")
nyc_data["year"]  = pd.DatetimeIndex(nyc_data["pickup_datetime"]).year
nyc_data["month"] = pd.DatetimeIndex(nyc_data["pickup_datetime"]).month
nyc_data["month_name"] = nyc_data["month"].map({1:"JAN",2:"FEB",3:"MAR",
                                                4:"APR",5:"MAY",6:"JUN",
                                                7:"JUL",8:"AUG",9:"SEP",
                                                10:"OCT",11:"NOV",12:"DEC"
                                               })
nyc_data["month_year"] = nyc_data["year"].astype(str) + " - " + nyc_data["month_name"]
nyc_data["week_day"]   = nyc_data["pickup_datetime"].dt.weekday_name
nyc_data["day"]        = nyc_data["pickup_datetime"].dt.day
nyc_data["hour"]        = nyc_data["pickup_datetime"].dt.hour 
nyc_data = nyc_data.sort_values(by = "pickup_datetime",ascending = False)

nyc_data = nyc_data[(nyc_data["passenger_count"] > 0 ) &
                    (nyc_data["passenger_count"] < 7) ]

nyc_data = nyc_data[ (nyc_data["fare_amount"] > 0 ) &
                     (nyc_data["fare_amount"]  <  
                      nyc_data["fare_amount"].quantile(.9999))]

coords = ['pickup_longitude','pickup_latitude', 
          'dropoff_longitude', 'dropoff_latitude']
for i in coord  : 
    nyc_data = nyc_data[(nyc_data[i]   > nyc_data[i].quantile(.001)) & 
                        (nyc_data[i] < nyc_data[i].quantile(.999))]
    
nyc_data["log_fare_amount"] = np.log(nyc_data["fare_amount"])
    
nyc_data.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,month_name,month_year,week_day,day,hour,log_fare_amount
785182,2015-06-30 23:53:49.0000003,7.5,2015-06-30 23:53:49,-73.959969,40.762405,-73.953064,40.782688,1,2015,6,JUN,2015 - JUN,Tuesday,30,23,2.014903
751350,2015-06-30 23:53:23.0000002,3.5,2015-06-30 23:53:23,-73.978020,40.757439,-73.980705,40.753544,1,2015,6,JUN,2015 - JUN,Tuesday,30,23,1.252763
915826,2015-06-30 23:48:35.0000005,30.5,2015-06-30 23:48:35,-73.983826,40.729546,-73.927917,40.661186,2,2015,6,JUN,2015 - JUN,Tuesday,30,23,3.417727
1369248,2015-06-30 23:48:17.0000004,8.5,2015-06-30 23:48:17,-73.978996,40.761471,-73.985603,40.755539,1,2015,6,JUN,2015 - JUN,Tuesday,30,23,2.140066
1028278,2015-06-30 23:47:07.0000003,11.5,2015-06-30 23:47:07,-73.979668,40.752071,-73.976433,40.785683,1,2015,6,JUN,2015 - JUN,Tuesday,30,23,2.442347


In [4]:
R = 6373.0

pickup_lat  = np.radians(nyc_data["pickup_latitude"])
pickup_lon  = np.radians(nyc_data["pickup_longitude"])
dropoff_lat = np.radians(nyc_data["dropoff_latitude"])
dropoff_lon = np.radians(nyc_data["dropoff_longitude"])

dist_lon = dropoff_lon - pickup_lon
dist_lat = dropoff_lat - pickup_lat

a = (np.sin(dist_lat/2))**2 + np.cos(pickup_lat) * np.cos(dropoff_lat) * (np.sin(dist_lon/2))**2 
c = 2 * np.arctan2( np.sqrt(a), np.sqrt(1-a) ) 
d = R * c 

nyc_data["trip_distance_km"] = d

nyc_data["log_trip_ditance"] = np.log(nyc_data["trip_distance_km"])

nyc_data[coord + ["trip_distance_km"]].head(7)

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_distance_km
785182,-73.959969,40.762405,-73.953064,40.782688,2.329810
751350,-73.978020,40.757439,-73.980705,40.753544,0.488753
915826,-73.983826,40.729546,-73.927917,40.661186,8.946793
1369248,-73.978996,40.761471,-73.985603,40.755539,0.863256
1028278,-73.979668,40.752071,-73.976433,40.785683,3.748497
955575,-74.002342,40.739819,-74.005829,40.745239,0.670727
1861371,-73.780014,40.647179,-73.992836,40.733597,20.361028


In [5]:
summary = nyc_data.describe().transpose().reset_index().rename(columns = {"index" : 
                                                                          "variable"})
summary  = np.around(summary,2)

var_lst = [summary["variable"],summary["count"],summary['mean'],summary['std'],
           summary["min"],summary["25%"],summary["50%"],summary["75%"],summary["max"]]

table = go.Table(header = dict(values = summary.columns.tolist(),
                               line = dict(color = ['#506784']),
                               fill = dict(color = ['#119DFF']),
                              ),
                 cells  = dict(values = var_lst,
                               line = dict(color = ['#506784']),
                               fill = dict(color = ["lightgrey",'#F5F8FF']),
                              ),
                 columnwidth = [130,80,80,80,80,80,80,80,80])
                
layout = go.Layout(dict(title = "Variable Summary"))
figure = go.Figure(data=[table],layout=layout)
py.iplot(figure)

In [6]:
trace = go.Pie(values = [nyc_data.shape[0],5000000 - nyc_data.shape[0]],
               labels = ["Available data" , "Data loss due to outliers and missing values"],
               marker = dict(colors =  [ 'royalblue' ,'lime'],line = dict(color = "black",
                                                                          width =  1.5)),
               rotation  = 60,
               hoverinfo = "label+percent",
              )

layout = go.Layout(dict(title = "Data Loss due to outliers and missing values",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                       )
                  )

fig = go.Figure(data=[trace],layout=layout)
py.iplot(fig)

In [7]:
def plot_day_trend(year) :
    day_count = complete_dat[complete_dat["year"] == year]["week_day"].value_counts().reset_index()
    day_count.columns = ["day","count"]
    day_count["order"]  = day_count["day"].replace({"Sunday" :1,'Monday' : 2, 'Tuesday': 3,
                                                    'Wednesday':4,'Thursday' :5, 'Friday':6,
                                                    'Saturday':7})
    day_count = day_count.sort_values(by = "order",ascending  = True)
    
    tracer = go.Bar(x = day_count["day"],y = day_count["count"],
                    name = year,marker = dict(line = dict(width =1))
                   )
    
    return tracer


layout = go.Layout(dict(title = "Trend in trips  by weekdays",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "weekday",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "count",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                       )
                  )

t  = plot_day_trend(2009)
t1 = plot_day_trend(2010)
t2 = plot_day_trend(2011)
t3 = plot_day_trend(2012)
t4 = plot_day_trend(2013)
t5 = plot_day_trend(2014)

data = [t,t1,t2,t3,t4,t5]
py.iplot(go.Figure(data = data,layout=layout))

NameError: name 'complete_dat' is not defined

In [ ]:
trips_hr = nyc_data["hour"].value_counts().reset_index()
trips_hr.columns = ["hour","count"]
trips_hr = trips_hr.sort_values(by = "hour",ascending = True)

trace = go.Scatter(x = trips_hr["hour"],y = trips_hr["count"],
                   mode = "markers+lines",
                  marker = dict(color = "red",size = 9,
                                line = dict(color = "black",width =2)))
#layout
layout = go.Layout(dict(title = "Trend in trips  by hour of day",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',title = "hour",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',title = "count",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                       )
                  )

fig = go.Figure(data = [trace],layout = layout)
py.iplot(fig)

In [ ]:
avg_fare_hr = complete_dat.groupby("hour")["fare_amount"].mean().reset_index()
avg_fare_hr
trace = go.Scatter(x = avg_fare_hr["hour"],y = avg_fare_hr["fare_amount"],
                   mode = "markers+lines",
                  marker = dict(color = "blue",size = 9,
                                line = dict(color = "black",width =2)))

#layout
layout = go.Layout(dict(title = "Average fare by hour",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',title = "hour",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',title = "average_fare",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                       )
                  )

fig = go.Figure(data = [trace],layout = layout)
py.iplot(fig)

In [ ]:
import datetime

ts_fare = total_fare.copy()
ts_fare["date"] = ts_fare["year"].astype(str) + "-" + ts_fare["month"].astype(str)

ts_fare = ts_fare[["date","fare_amount"]]

ts_fare["date"] = pd.to_datetime(ts_fare["date"],format = "%Y-%m")
ts_fare.index   = ts_fare["date"]
ts_fare = ts_fare.drop(columns  = ["date"],axis = 1)
ts_fare.head(10)

In [ ]:
trace = go.Scatter(x = ts_fare.index,y = ts_fare.fare_amount,
                   mode = "lines+markers",
                   marker = dict(color = "royalblue",line = dict(width =1))
                  )
layout = go.Layout(dict(title = "Visualizing time series",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',title = "count",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        margin = dict(b = 100)
                       )
                  )
fig = go.Figure(data = [trace],layout = layout)
py.iplot(fig)

In [ ]:
from statsmodels.tsa.stattools import adfuller

def plot_line(x,y,color,name) :
    tracer = go.Scatter(x = x,y = y,mode = "lines",
                        marker = dict(color = color,
                                      line = dict(width =1)),
                       name = name)
    return tracer

def plot_layout(title) :
    layout = go.Layout(dict(title = title,
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                         zerolinewidth=1,ticklen=5,gridwidth=2),
                            yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                            zerolinewidth=1,ticklen=5,gridwidth=2),
                        margin = dict(b = 100)
                       )
                  )
    return layout


def stationary_test(timeseries) :
    #rolling mean
    rol_mean = timeseries["fare_amount"].rolling(window = 12,
                                                 center = False).mean()
    #rolling standard deviation
    rol_std  = timeseries["fare_amount"].rolling(window = 12,
                                                 center = False).std()
    
    #plotting
    trace1  = plot_line(timeseries.index,timeseries["fare_amount"],
                        "blue","time_series")
    trace2  = plot_line(rol_mean.index,rol_mean.values,
                        "red","rolling_mean")
    trace3  = plot_line(rol_std.index,rol_std.values,
                        "green", "rolling_std")
    layout  = plot_layout("rolling mean and standard deviation for timeseries")
    figure  = go.Figure(data = [trace1,trace2,trace3],layout = layout)
    
    test_results = adfuller(timeseries["fare_amount"])
    res_list     = ["Test Statistic","p-value",
                    "lags used","no of observations"] 
    res_df = pd.Series(test_results[:4],index = res_list)
    
    for key,value in test_results[4].items() :
        res_df["Critical value (%s)"%key] = value 
        
    print ("Results - Dickey fuller test")
    print (res_df)
    return py.iplot(figure)

stationary_test(ts_fare)

In [ ]:
#log of timeseries
log_ts_fare = np.log(ts_fare)

#rolling average of log timeseries
rol_avg_log_ts = log_ts_fare["fare_amount"].rolling(window = 12,center = False).mean()

#plotting log timeseries and rolling mean
t1 = plot_line(log_ts_fare.index,log_ts_fare.fare_amount,
                "blue","log_time_series")
t2 = plot_line(rol_avg_log_ts.index,rol_avg_log_ts.values,
               "red","moving_average(log)")
lay = plot_layout("log time series and moving average")
fig = go.Figure(data = [t1,t2],layout = lay)
py.iplot(fig)

#difference
log_ts_fare_diff = log_ts_fare - rol_avg_log_ts.to_frame()
log_ts_fare_diff.dropna(inplace = True)
stationary_test(log_ts_fare_diff)

In [ ]:
#exponential moving average of log time series
exp_log_avg = log_ts_fare["fare_amount"].ewm(halflife = 12).mean()

#plotting
t1 = plot_line(log_ts_fare.index,log_ts_fare["fare_amount"],
               "blue","log time series")
t2 = plot_line(exp_log_avg.index,exp_log_avg.values,
               "red","exponential avg")
lay = plot_layout("log time series and exponential moving average")
fig = go.Figure(data = [t1,t2],layout = lay)
py.iplot(fig)

#difference
exp_ts_diff = log_ts_fare - exp_log_avg.to_frame()
stationary_test(exp_ts_diff)

In [ ]:
#differencing log series
ts_fare_diff = log_ts_fare - log_ts_fare.shift()
ts_fare_diff.dropna(inplace = True)

#plotting
t1 = plot_line(ts_fare_diff.index,ts_fare_diff["fare_amount"],
              "blue","Differenced log series")
lay = plot_layout("Differenced log series")
fig = go.Figure(data = [t1],layout=lay)
py.iplot(fig)

#stationary test
stationary_test(ts_fare_diff)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

#decompose
decompose = seasonal_decompose(log_ts_fare)

#trend
trend       = decompose.trend
#seasonality
seasonality = decompose.seasonal
#residuals
residuals   = decompose.resid

#plotting
t1 = plot_line(ts_fare.index,ts_fare.fare_amount,
               "blue","log_Series")
t2 = plot_line(trend.index,trend.fare_amount,
               "green","Trend")
t3 = plot_line(seasonality.index,seasonality.fare_amount,
               "red","Seasonality")
t4 = plot_line(residuals.index,residuals.fare_amount,
               "black","Residuals")
#subplots
fig = tls.make_subplots(rows = 4,cols = 1,subplot_titles = ("log series",
                                                            "Trend",
                                                            "Seasonality",
                                                            "residuals"))

fig.append_trace(t1,1,1)
fig.append_trace(t2,2,1)
fig.append_trace(t3,3,1)
fig.append_trace(t4,4,1)
#layout
fig["layout"].update(height = 750,
                     plot_bgcolor  = "rgb(243,243,243)",
                     paper_bgcolor = "rgb(243,243,243)",
                     xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                  zerolinewidth=1,ticklen=5,gridwidth=2),
                    yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                 zerolinewidth=1,ticklen=5,gridwidth=2),
                    title = "decomposing"
                    )
py.iplot(fig)

#stationary tert for residuals
residuals.dropna(inplace = True)
stationary_test(residuals)

In [ ]:
from statsmodels.tsa.stattools import acf,pacf

#auto correlation function
acf_vals  = acf(ts_fare_diff)
#partial auto correlation function
pacf_vals = pacf(ts_fare_diff)

#plot acf,pacf
def plot_corr_fact(x,y,color,name) :
    tracer = go.Bar(x = x, y= y ,
                    marker = dict(color = color,
                                  line = dict(width =1,
                                              color = "black") 
                                 ),
                    name = name
                   )
    return tracer
#plot confidence intervals
def plot_lines(x,y) :
    trace_line = go.Scatter(x = x, y = y,
                            mode   = "lines",
                            line   = dict(color = "black",
                                          width = 2,
                                          dash = "dash" 
                                         ) ,
                            name = "confidence intervals"
                           )
    return trace_line

#acf values
t_acf  = plot_corr_fact(np.arange(0,len(acf_vals)),
                    acf_vals,"blue","acf")
#confidence intervals for acf
lu_acf = plot_lines(np.arange(0,len(acf_vals)),
                    [1.96/np.sqrt(len(ts_fare_diff))]*len(acf_vals))
ll_acf = plot_lines(np.arange(0,len(acf_vals)),
                    [-1.96/np.sqrt(len(ts_fare_diff))]*len(acf_vals))

#pacf values
t_pacf = plot_corr_fact(np.arange(0,len(pacf_vals)),
                    pacf_vals,"red","pacf")
#confidence intervals for pacf
lu_pacf = plot_lines(np.arange(0,len(pacf_vals)),
                    [1.96/np.sqrt(len(ts_fare_diff))]*len(pacf_vals))
ll_pacf = plot_lines(np.arange(0,len(pacf_vals)),
                    [-1.96/np.sqrt(len(ts_fare_diff))]*len(pacf_vals))

#subplots
fig = tls.make_subplots(rows = 1, cols  = 2,
                        subplot_titles = ("auto correlation function",
                                          "partial auto correlation function"))

fig.append_trace(t_acf,1,1)
fig.append_trace(lu_acf,1,1)
fig.append_trace(ll_acf,1,1)
fig.append_trace(t_pacf,1,2)
fig.append_trace(lu_pacf,1,2)
fig.append_trace(ll_pacf,1,2)

#layout
fig["layout"].update(plot_bgcolor  = "rgb(243,243,243)",
                     showlegend = False,
                     paper_bgcolor = "rgb(243,243,243)",
                     xaxis1 = dict(gridcolor = 'rgb(255, 255, 255)',
                                  zerolinewidth=1,ticklen=5,gridwidth=2),
                     yaxis1 = dict(gridcolor = 'rgb(255, 255, 255)',
                                 zerolinewidth=1,ticklen=5,gridwidth=2),
                     xaxis2 = dict(gridcolor = 'rgb(255, 255, 255)',
                                  zerolinewidth=1,ticklen=5,gridwidth=2),
                     yaxis2 = dict(gridcolor = 'rgb(255, 255, 255)',
                                 zerolinewidth=1,ticklen=5,gridwidth=2))


py.iplot(fig)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

#ARIMA model
def arima_model(time_series,p,d,q) :
    arima_model   = ARIMA(time_series , order = (p,d,q))
    results_arima = arima_model.fit(disp = -1)
    fitted_values = results_arima.fittedvalues
    
    trace1 = plot_line(fitted_values.index,
                       fitted_values.values,
                       "blue","fitted values")
    
    trace2 = plot_line(ts_fare_diff.index,
                       ts_fare_diff["fare_amount"],
                       "red","log differenced values")

    layout = plot_layout(("ARIMA model p = " + str(p) + 
                          ", d = " + str(d) + ", q = " + str(q)))
    data  = [trace2,trace1]
    fig   = go.Figure(data = data,layout = layout)
    py.iplot(fig)
    print (results_arima.summary())
    
arima_model(log_ts_fare,1,1,0)


In [ ]:
arima_model(log_ts_fare,1,1,1)


In [ ]:
arima_model(log_ts_fare,2,1,2)


Summary:
    Arima Model, Autoregressive Integrated Moving Average Model.
    I have taken p, d and q as {1,1,0} , {1,1,1} and {2,1,2} respectively.  
    Out of which the {2,1,2} works well as compared to the {1,1,1} and {1,1,0}.
    